In [ ]:
import sys
import numpy as np
from pathlib import Path

# FFD 모듈 import
sys.path.append('scripts')
from ffd_airfoil import FFDAirfoil, load_airfoil, save_airfoil, generate_naca_baseline, generate_random_deformation

# Plotting (optional)
try:
    import matplotlib.pyplot as plt
    %matplotlib inline
    plt.rcParams['figure.figsize'] = (12, 6)
    HAS_MATPLOTLIB = True
except ImportError:
    print("⚠️ matplotlib not available - visualization disabled")
    HAS_MATPLOTLIB = False

## 1. Baseline Airfoil 생성

먼저 NACA airfoil을 baseline으로 생성합니다.

In [ ]:
# NACA 0012 airfoil 생성
naca_code = "0012"
baseline_coords = generate_naca_baseline(naca_code, output_file="temp_baseline.dat")

if baseline_coords is not None:
    print(f"✅ NACA {naca_code} baseline generated")
    print(f"   Points: {len(baseline_coords)}")
    print(f"   X range: [{baseline_coords[:, 0].min():.4f}, {baseline_coords[:, 0].max():.4f}]")
    print(f"   Y range: [{baseline_coords[:, 1].min():.4f}, {baseline_coords[:, 1].max():.4f}]")
else:
    print("❌ Failed to generate baseline")

## 2. FFD 초기화

제어점 격자를 설정합니다. 
- NX = 5: chord 방향으로 5개 제어점
- NY = 3: 두께 방향으로 3개 제어점 (윗면, 중간, 아랫면)

In [ ]:
# FFD 초기화
n_control_x = 5
n_control_y = 3

ffd = FFDAirfoil(n_control_x=n_control_x, n_control_y=n_control_y)
ffd.setup_lattice(baseline_coords, padding=0.15)

print(f"✅ FFD lattice initialized: {n_control_x} × {n_control_y} control points")
print(f"   Bounding box: x=[{ffd.bbox[0]:.3f}, {ffd.bbox[1]:.3f}], y=[{ffd.bbox[2]:.3f}, {ffd.bbox[3]:.3f}]")

## 3. 제어점 시각화

초기 제어점 격자와 airfoil을 함께 시각화합니다.

In [ ]:
if HAS_MATPLOTLIB:
    fig, ax = plt.subplots(figsize=(14, 7))
    
    # Airfoil
    ax.plot(baseline_coords[:, 0], baseline_coords[:, 1], 'b-', linewidth=2, label='NACA 0012 Baseline')
    
    # Control points
    cp_x = ffd.control_points[:, :, 0]
    cp_y = ffd.control_points[:, :, 1]
    
    # Draw control lattice
    for i in range(n_control_x):
        ax.plot(cp_x[i, :], cp_y[i, :], 'g--', alpha=0.4, linewidth=1)
    for j in range(n_control_y):
        ax.plot(cp_x[:, j], cp_y[:, j], 'g--', alpha=0.4, linewidth=1)
    
    # Control points
    ax.plot(cp_x, cp_y, 'go', markersize=10, label='Control Points', zorder=5)
    
    # Label control points
    for i in range(n_control_x):
        for j in range(n_control_y):
            ax.text(cp_x[i, j], cp_y[i, j] + 0.02, f'({i},{j})', 
                   ha='center', va='bottom', fontsize=8, color='darkgreen')
    
    ax.set_xlabel('x/c', fontsize=12)
    ax.set_ylabel('y/c', fontsize=12)
    ax.set_title(f'FFD Control Lattice ({n_control_x}×{n_control_y})', fontsize=14, fontweight='bold')
    ax.axis('equal')
    ax.grid(True, alpha=0.3)
    ax.legend(fontsize=11)
    plt.tight_layout()
    plt.show()
else:
    print("Visualization skipped (matplotlib not available)")

## 4. 랜덤 변형 적용

제어점에 랜덤 변형을 적용하여 새로운 airfoil 형상을 생성합니다.

In [ ]:
# 랜덤 변형 생성 (amplitude = chord의 2%)
amplitude = 0.02
deformation = generate_random_deformation(n_control_x, n_control_y, amplitude=amplitude)

print(f"Generated random deformation:")
print(f"  Shape: {deformation.shape}")
print(f"  Min displacement: {deformation.min():.6f}")
print(f"  Max displacement: {deformation.max():.6f}")
print(f"  Mean displacement: {deformation.mean():.6f}")

# 변형 적용
ffd.apply_deformation(deformation)
print(f"\n✅ Deformation applied to control points")

## 5. Airfoil 변형 및 비교

FFD를 사용하여 airfoil을 변형하고 원본과 비교합니다.

In [ ]:
# Airfoil 변형
deformed_coords = ffd.deform_airfoil(baseline_coords)

print(f"✅ Airfoil deformed")
print(f"   Points: {len(deformed_coords)}")
print(f"   X range: [{deformed_coords[:, 0].min():.4f}, {deformed_coords[:, 0].max():.4f}]")
print(f"   Y range: [{deformed_coords[:, 1].min():.4f}, {deformed_coords[:, 1].max():.4f}]")

# 변형 정도 계산
displacement = np.linalg.norm(deformed_coords - baseline_coords, axis=1)
print(f"\nDisplacement statistics:")
print(f"   Mean: {displacement.mean():.6f}")
print(f"   Max:  {displacement.max():.6f}")
print(f"   Min:  {displacement.min():.6f}")

In [ ]:
if HAS_MATPLOTLIB:
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Left plot: Comparison
    ax = axes[0]
    ax.plot(baseline_coords[:, 0], baseline_coords[:, 1], 'b-', linewidth=2.5, label='Original', alpha=0.7)
    ax.plot(deformed_coords[:, 0], deformed_coords[:, 1], 'r-', linewidth=2.5, label='FFD Deformed', alpha=0.7)
    
    # Plot deformed control points
    cp_x_def = ffd.control_points[:, :, 0]
    cp_y_def = ffd.control_points[:, :, 1]
    ax.plot(cp_x_def, cp_y_def, 'go', markersize=8, label='Deformed Control Points', alpha=0.6)
    
    ax.set_xlabel('x/c', fontsize=12)
    ax.set_ylabel('y/c', fontsize=12)
    ax.set_title('Original vs FFD Deformed Airfoil', fontsize=13, fontweight='bold')
    ax.axis('equal')
    ax.grid(True, alpha=0.3)
    ax.legend(fontsize=10)
    
    # Right plot: Displacement field
    ax = axes[1]
    scatter = ax.scatter(baseline_coords[:, 0], baseline_coords[:, 1], 
                        c=displacement, cmap='hot', s=20, alpha=0.8)
    
    # Draw displacement vectors (sample every 10 points)
    step = max(1, len(baseline_coords) // 30)
    for i in range(0, len(baseline_coords), step):
        dx = deformed_coords[i, 0] - baseline_coords[i, 0]
        dy = deformed_coords[i, 1] - baseline_coords[i, 1]
        ax.arrow(baseline_coords[i, 0], baseline_coords[i, 1], dx, dy, 
                head_width=0.01, head_length=0.01, fc='blue', ec='blue', alpha=0.5, linewidth=0.5)
    
    plt.colorbar(scatter, ax=ax, label='Displacement')
    ax.set_xlabel('x/c', fontsize=12)
    ax.set_ylabel('y/c', fontsize=12)
    ax.set_title('Displacement Field', fontsize=13, fontweight='bold')
    ax.axis('equal')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("Visualization skipped (matplotlib not available)")

## 6. 결과 저장

변형된 airfoil을 파일로 저장합니다.

In [ ]:
# 출력 디렉토리 생성
output_dir = Path("public/airfoil")
output_dir.mkdir(parents=True, exist_ok=True)

# 저장
output_file = output_dir / "ffd_example.dat"
save_airfoil(str(output_file), deformed_coords, f"FFD NACA {naca_code} Example")

print(f"\n✅ Deformed airfoil saved to: {output_file.absolute()}")

## 7. 다중 샘플 생성 (Surrogate Model용)

여러 개의 랜덤 샘플을 생성하여 surrogate model 학습용 데이터셋을 만듭니다.

In [ ]:
# 설정
n_samples = 10
sample_dir = Path("samples/tutorial_ffd")
sample_dir.mkdir(parents=True, exist_ok=True)

# 파라미터 저장용 리스트
all_parameters = []

print(f"Generating {n_samples} FFD samples...")
print("=" * 50)

for i in range(n_samples):
    # FFD 재초기화
    ffd_sample = FFDAirfoil(n_control_x=n_control_x, n_control_y=n_control_y)
    ffd_sample.setup_lattice(baseline_coords, padding=0.15)
    
    # 랜덤 변형 생성 및 적용
    deform = generate_random_deformation(n_control_x, n_control_y, amplitude=amplitude)
    ffd_sample.apply_deformation(deform)
    
    # Airfoil 변형
    sample_coords = ffd_sample.deform_airfoil(baseline_coords)
    
    # 저장
    output_file = sample_dir / f"ffd_sample_{i:03d}.dat"
    save_airfoil(str(output_file), sample_coords, f"FFD Sample {i:03d}")
    
    # 파라미터 저장
    all_parameters.append(deform.flatten())

# 파라미터 저장
param_file = sample_dir / "deformation_parameters.txt"
np.savetxt(param_file, np.array(all_parameters))

print(f"\n✅ Generated {n_samples} samples in: {sample_dir.absolute()}")
print(f"✅ Saved parameters to: {param_file.absolute()}")

## 8. 샘플 시각화

생성된 모든 샘플을 한 번에 시각화합니다.

In [ ]:
if HAS_MATPLOTLIB:
    fig, ax = plt.subplots(figsize=(14, 8))
    
    # Baseline
    ax.plot(baseline_coords[:, 0], baseline_coords[:, 1], 'k-', 
           linewidth=3, label='Baseline', zorder=10, alpha=0.8)
    
    # Load and plot all samples
    sample_files = sorted(sample_dir.glob("ffd_sample_*.dat"))
    colors = plt.cm.viridis(np.linspace(0, 1, len(sample_files)))
    
    for i, sample_file in enumerate(sample_files):
        coords, _ = load_airfoil(str(sample_file))
        ax.plot(coords[:, 0], coords[:, 1], '-', 
               color=colors[i], linewidth=1.5, alpha=0.6, label=f'Sample {i}')
    
    ax.set_xlabel('x/c', fontsize=12)
    ax.set_ylabel('y/c', fontsize=12)
    ax.set_title(f'FFD Sample Dataset ({n_samples} samples)', fontsize=14, fontweight='bold')
    ax.axis('equal')
    ax.grid(True, alpha=0.3)
    
    # Legend with fewer entries for clarity
    handles, labels = ax.get_legend_handles_labels()
    if len(handles) > 6:
        ax.legend([handles[0]], [labels[0]], fontsize=11)  # Just show baseline
    else:
        ax.legend(fontsize=9, ncol=2)
    
    plt.tight_layout()
    plt.show()
else:
    print("Visualization skipped (matplotlib not available)")

## 9. 다음 단계

생성된 FFD 샘플들로 할 수 있는 작업:

### 9.1 XFOIL 해석 수행
```bash
# 각 샘플에 대해 AoA sweep
for f in samples/tutorial_ffd/ffd_sample_*.dat; do
    python scripts/aoa_sweep.py "$f" 1000000 -5 15 0.5
done
```

### 9.2 Surrogate Model 학습
- 변형 파라미터 (입력): deformation_parameters.txt
- 공력 성능 (출력): aoa_sweep 결과 (CL, CD, CM 등)
- 머신러닝 모델: Neural Network, Gaussian Process, Kriging 등

### 9.3 최적화
- Surrogate model을 사용하여 빠른 형상 최적화
- Genetic Algorithm, Gradient-based optimization 등 적용

### 9.4 설계 공간 탐색
- 파라미터 민감도 분석
- 다목적 최적화 (Multi-objective: max L/D, min CD 등)

## 요약

이 튜토리얼에서 배운 내용:

1. ✅ FFD를 사용한 airfoil 매개변수화
2. ✅ 제어점 격자 설정 및 시각화
3. ✅ 랜덤/특정 변형 적용
4. ✅ 다중 샘플 생성 (surrogate model용)
5. ✅ 변형 결과 시각화 및 저장

### 핵심 파라미터:
- **제어점 개수 (NX, NY)**: 형상 자유도 조절
- **변형 크기 (amplitude)**: 변형 정도 제어 (권장: 0.01~0.03)
- **샘플 개수**: surrogate model 정확도에 영향

### 권장 워크플로우:
1. 적절한 baseline airfoil 선택 (NACA 또는 기존 설계)
2. FFD 제어점 개수 결정 (일반적으로 5×3)
3. 다양한 샘플 생성 (100~1000개)
4. XFOIL 해석 수행
5. Surrogate model 학습
6. 최적화 수행